In [ ]:
"""

COURSE: Artificial Intelligence II
Week 4 Assignment: Markov Decision Process: Environment Navigation Model Using Policy Iteration
SEMESTER: Fall 2021
NAME: Joe Cruz
DATE: 26Nov2021

The purpose of this algorithm is to simulate agent navigation through a simulated environment 
using Markov Decision Process (MDP) based algorithm in tandem with policy iteration. Additionally,
a time study is performed using differently scaled environments.


"""

In [2]:
"""
THIS BLOCK OF CODE IS FROM THE FOLLOWING REFERENCE:
P. Norvig, et al., “mdp.py” [Online]. Available: https://github.com/aimacode/aima-python/blob/master/mdp.py. [Accessed: November 24, 2021]. [Source Code]. 


This was used and slightly modified to display utility values when the policy iteration is performed. 
Any changes made by Joe Cruz have 7 #'s and a comment regarding the change made. Otherwise the rest
of this code was from the reference mentioned at the top.

######

Markov Decision Processes (Chapter 17)
First we define an MDP, and the special case of a GridMDP, in which
states are laid out in a 2-dimensional grid. We also represent a policy
as a dictionary of {state: action} pairs, and a Utility function as a
dictionary of {state: number} pairs. We then define the value_iteration
and policy_iteration algorithms.
"""

import random
from collections import defaultdict

import numpy as np

from utils import vector_add, orientations, turn_right, turn_left, print_table


class MDP:
    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state. [Page 646]"""

    def __init__(self, init, actlist, terminals, transitions=None, reward=None, states=None, gamma=0.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        # collect states from transitions table if not passed.
        self.states = states or self.get_states_from_transitions(transitions)

        self.init = init

        if isinstance(actlist, list):
            # if actlist is a list, all states have the same actions
            self.actlist = actlist

        elif isinstance(actlist, dict):
            # if actlist is a dict, different actions for each state
            self.actlist = actlist

        self.terminals = terminals
        self.transitions = transitions or {}
        if not self.transitions:
            print("Warning: Transition table is empty.")

        self.gamma = gamma

        self.reward = reward or {s: 0 for s in self.states}

        # self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""

        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""

        if not self.transitions:
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set(tr[1] for actions in transitions.values()
                     for effects in actions.values()
                     for tr in effects)
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):

        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)

        # check that init is a valid state
        assert self.init in self.states

        # check reward for each state
        assert set(self.reward.keys()) == set(self.states)

        # check that all terminals are valid states
        assert all(t in self.states for t in self.terminals)

        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001


class MDP2(MDP):
    """
    Inherits from MDP. Handles terminal states, and transitions to and from terminal states better.
    """

    def __init__(self, init, actlist, terminals, transitions, reward=None, gamma=0.9):
        MDP.__init__(self, init, actlist, terminals, transitions, reward, gamma=gamma)

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return self.transitions[state][action]


class GridMDP(MDP):
    """A two-dimensional grid MDP, as in [Figure 17.1]. All you have to do is
    specify the grid as a list of lists of rewards; use None for an obstacle
    (unreachable state). Also, you should specify the terminal states.
    An action is an (x, y) unit vector; e.g. (1, 0) means move east."""

    def __init__(self, grid, terminals, init=(0, 0), gamma=.9):
        grid.reverse()  # because we want row 0 on bottom, not on top
        reward = {}
        states = set()
        self.rows = len(grid)
        self.cols = len(grid[0])
        self.grid = grid
        for x in range(self.cols):
            for y in range(self.rows):
                if grid[y][x]:
                    states.add((x, y))
                    reward[(x, y)] = grid[y][x]
        self.states = states
        actlist = orientations
        transitions = {}
        for s in states:
            transitions[s] = {}
            for a in actlist:
                transitions[s][a] = self.calculate_T(s, a)
        MDP.__init__(self, init, actlist=actlist,
                     terminals=terminals, transitions=transitions,
                     reward=reward, states=states, gamma=gamma)

    def calculate_T(self, state, action):
        if action:
            return [(0.8, self.go(state, action)),
                    (0.1, self.go(state, turn_right(action))),
                    (0.1, self.go(state, turn_left(action)))]
        else:
            return [(0.0, state)]

    def T(self, state, action):
        return self.transitions[state][action] if action else [(0.0, state)]

    def go(self, state, direction):
        """Return the state that results from going in this direction."""

        state1 = vector_add(state, direction)
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        """Convert a mapping from (x, y) to v into a [[..., v, ...]] grid."""

        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

    def to_arrows(self, policy):
        chars = {(1, 0): '>', (0, 1): '^', (-1, 0): '<', (0, -1): 'v', None: '.'}
        return self.to_grid({s: chars[a] for (s, a) in policy.items()})


# ______________________________________________________________________________


""" [Figure 17.1]
A 4x3 grid environment that presents the agent with a sequential decision problem.
"""
#4x3
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, +1],
                                           [ -0.04, None, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(3, 2), (3, 1)])
# ______________________________________________________________________________


def value_iteration(mdp, epsilon=0.001):
    """Solving an MDP by value iteration. [Figure 17.4]"""

    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max(sum(p * U[s1] for (p, s1) in T(s, a))
                                       for a in mdp.actions(s))
            delta = max(delta, abs(U1[s] - U[s]))
        if delta <= epsilon * (1 - gamma) / gamma:
            return U


def best_policy(mdp, U):
    """Given an MDP and a utility function U, determine the best policy,
    as a mapping from state to action. [Equation 17.4]"""

    pi = {}
    for s in mdp.states:
        pi[s] = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
    return pi


def expected_utility(a, s, U, mdp):
    """The expected utility of doing a in state s, according to the MDP and U."""

    return sum(p * U[s1] for (p, s1) in mdp.T(s, a))


# ______________________________________________________________________________


def policy_iteration(mdp):
    """Solve an MDP by policy iteration [Figure 17.7]"""

    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
        #print(U)
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            #######CHANGE MADE HERE: ADDED SEGMENT TO MAP THE UTILITY VALUES FOR EACH STATE TO 
            #######THEIR RESPECTIVE POSITIONS IN THE MATRIX ALSO MADE THEM PRINTED WHEN PROGRAM
            #######IS RUN.
            print("Computed utilities for this policy iteration:")
            grid_U = mdp.to_grid(U)
            for entry in grid_U:
                print(entry)
            return pi


def policy_evaluation(pi, U, mdp, k=20):
    """Return an updated utility mapping U from each state in the MDP to its
    utility, using an approximation (modified policy iteration)."""

    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum(p * U[s1] for (p, s1) in T(s, pi[s]))
    return U


class POMDP(MDP):
    """A Partially Observable Markov Decision Process, defined by
    a transition model P(s'|s,a), actions A(s), a reward function R(s),
    and a sensor model P(e|s). We also keep track of a gamma value,
    for use by algorithms. The transition and the sensor models
    are defined as matrices. We also keep track of the possible states
    and actions for each state. [Page 659]."""

    def __init__(self, actions, transitions=None, evidences=None, rewards=None, states=None, gamma=0.95):
        """Initialize variables of the pomdp"""

        if not (0 < gamma <= 1):
            raise ValueError('A POMDP must have 0 < gamma <= 1')

        self.states = states
        self.actions = actions

        # transition model cannot be undefined
        self.t_prob = transitions or {}
        if not self.t_prob:
            print('Warning: Transition model is undefined')

        # sensor model cannot be undefined
        self.e_prob = evidences or {}
        if not self.e_prob:
            print('Warning: Sensor model is undefined')

        self.gamma = gamma
        self.rewards = rewards

    def remove_dominated_plans(self, input_values):
        """
        Remove dominated plans.
        This method finds all the lines contributing to the
        upper surface and removes those which don't.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = [values[0]]
        y1_max = max(val[1] for val in values)
        tgt = values[0]
        prev_b = 0
        prev_ix = 0
        while tgt[1] != y1_max:
            min_b = 1
            min_ix = 0
            for i in range(prev_ix + 1, len(values)):
                if values[i][0] - tgt[0] + tgt[1] - values[i][1] != 0:
                    trans_b = (values[i][0] - tgt[0]) / (values[i][0] - tgt[0] + tgt[1] - values[i][1])
                    if 0 <= trans_b <= 1 and trans_b > prev_b and trans_b < min_b:
                        min_b = trans_b
                        min_ix = i
            prev_b = min_b
            prev_ix = min_ix
            tgt = values[min_ix]
            best.append(tgt)

        return self.generate_mapping(best, input_values)

    def remove_dominated_plans_fast(self, input_values):
        """
        Remove dominated plans using approximations.
        Resamples the upper boundary at intervals of 100 and
        finds the maximum values at these points.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = []
        sr = 100
        for i in range(sr + 1):
            x = i / float(sr)
            maximum = (values[0][1] - values[0][0]) * x + values[0][0]
            tgt = values[0]
            for value in values:
                val = (value[1] - value[0]) * x + value[0]
                if val > maximum:
                    maximum = val
                    tgt = value

            if all(any(tgt != v) for v in best):
                best.append(np.array(tgt))

        return self.generate_mapping(best, input_values)

    def generate_mapping(self, best, input_values):
        """Generate mappings after removing dominated plans"""

        mapping = defaultdict(list)
        for value in best:
            for action in input_values:
                if any(all(value == v) for v in input_values[action]):
                    mapping[action].append(value)

        return mapping

    def max_difference(self, U1, U2):
        """Find maximum difference between two utility mappings"""

        for k, v in U1.items():
            sum1 = 0
            for element in U1[k]:
                sum1 += sum(element)
            sum2 = 0
            for element in U2[k]:
                sum2 += sum(element)
        return abs(sum1 - sum2)


class Matrix:
    """Matrix operations class"""

    @staticmethod
    def add(A, B):
        """Add two matrices A and B"""

        res = []
        for i in range(len(A)):
            row = []
            for j in range(len(A[0])):
                row.append(A[i][j] + B[i][j])
            res.append(row)
        return res

    @staticmethod
    def scalar_multiply(a, B):
        """Multiply scalar a to matrix B"""

        for i in range(len(B)):
            for j in range(len(B[0])):
                B[i][j] = a * B[i][j]
        return B

    @staticmethod
    def multiply(A, B):
        """Multiply two matrices A and B element-wise"""

        matrix = []
        for i in range(len(B)):
            row = []
            for j in range(len(B[0])):
                row.append(B[i][j] * A[j][i])
            matrix.append(row)

        return matrix

    @staticmethod
    def matmul(A, B):
        """Inner-product of two matrices"""

        return [[sum(ele_a * ele_b for ele_a, ele_b in zip(row_a, col_b)) for col_b in list(zip(*B))] for row_a in A]

    @staticmethod
    def transpose(A):
        """Transpose a matrix"""

        return [list(i) for i in zip(*A)]


def pomdp_value_iteration(pomdp, epsilon=0.1):
    """Solving a POMDP by value iteration."""

    U = {'': [[0] * len(pomdp.states)]}
    count = 0
    while True:
        count += 1
        prev_U = U
        values = [val for action in U for val in U[action]]
        value_matxs = []
        for i in values:
            for j in values:
                value_matxs.append([i, j])

        U1 = defaultdict(list)
        for action in pomdp.actions:
            for u in value_matxs:
                u1 = Matrix.matmul(Matrix.matmul(pomdp.t_prob[int(action)],
                                                 Matrix.multiply(pomdp.e_prob[int(action)], Matrix.transpose(u))),
                                   [[1], [1]])
                u1 = Matrix.add(Matrix.scalar_multiply(pomdp.gamma, Matrix.transpose(u1)), [pomdp.rewards[int(action)]])
                U1[action].append(u1[0])

        U = pomdp.remove_dominated_plans_fast(U1)
        # replace with U = pomdp.remove_dominated_plans(U1) for accurate calculations

        if count > 10:
            if pomdp.max_difference(U, prev_U) < epsilon * (1 - pomdp.gamma) / pomdp.gamma:
                return U

#__doc__ += 
"""
>>> pi = best_policy(sequential_decision_environment, value_iteration(sequential_decision_environment, .01))
>>> sequential_decision_environment.to_arrows(pi)
[['>', '>', '>', '.'], ['^', None, '^', '.'], ['^', '>', '^', '<']]
>>> from utils import print_table
>>> print_table(sequential_decision_environment.to_arrows(pi))
>   >      >   .
^   None   ^   .
^   >      ^   <
>>> print_table(sequential_decision_environment.to_arrows(policy_iteration(sequential_decision_environment)))
>   >      >   .
^   None   ^   .
^   >      ^   <
"""  # noqa

"""
s = { 'a' : {	'plan1' : [(0.2, 'a'), (0.3, 'b'), (0.3, 'c'), (0.2, 'd')],
                'plan2' : [(0.4, 'a'), (0.15, 'b'), (0.45, 'c')],
                'plan3' : [(0.2, 'a'), (0.5, 'b'), (0.3, 'c')],
                 },
      'b' : {	'plan1' : [(0.2, 'a'), (0.6, 'b'), (0.2, 'c'), (0.1, 'd')],
                'plan2' : [(0.6, 'a'), (0.2, 'b'), (0.1, 'c'), (0.1, 'd')],
                'plan3' : [(0.3, 'a'), (0.3, 'b'), (0.4, 'c')],
                },
        'c' : {	'plan1' : [(0.3, 'a'), (0.5, 'b'), (0.1, 'c'), (0.1, 'd')],
                'plan2' : [(0.5, 'a'), (0.3, 'b'), (0.1, 'c'), (0.1, 'd')],
                'plan3' : [(0.1, 'a'), (0.3, 'b'), (0.1, 'c'), (0.5, 'd')],
                },
    }
"""


"\ns = { 'a' : {\t'plan1' : [(0.2, 'a'), (0.3, 'b'), (0.3, 'c'), (0.2, 'd')],\n                'plan2' : [(0.4, 'a'), (0.15, 'b'), (0.45, 'c')],\n                'plan3' : [(0.2, 'a'), (0.5, 'b'), (0.3, 'c')],\n                 },\n      'b' : {\t'plan1' : [(0.2, 'a'), (0.6, 'b'), (0.2, 'c'), (0.1, 'd')],\n                'plan2' : [(0.6, 'a'), (0.2, 'b'), (0.1, 'c'), (0.1, 'd')],\n                'plan3' : [(0.3, 'a'), (0.3, 'b'), (0.4, 'c')],\n                },\n        'c' : {\t'plan1' : [(0.3, 'a'), (0.5, 'b'), (0.1, 'c'), (0.1, 'd')],\n                'plan2' : [(0.5, 'a'), (0.3, 'b'), (0.1, 'c'), (0.1, 'd')],\n                'plan3' : [(0.1, 'a'), (0.3, 'b'), (0.1, 'c'), (0.5, 'd')],\n                },\n    }\n"

In [3]:
#time package imported for measuring time for policy iteration run.
import time

#start time initiated for the policy iteration run
start = time.time()

#policy_iteration is performed to determine the optimal policy for the given environment. 
pi2 = policy_iteration(sequential_decision_environment)

#end time initiated for the policy iteration run.
end_time = time.time()

#Prints out the optimal policy determined by the policy iteration in an arrow/graph form.
#The arrows are the suggested direction to move from that state and the periods are the terminal or 
#end of navigation.
print("\nThis is the optimal policy in arrow/graph format: ")
print_table(sequential_decision_environment.to_arrows(pi2))

#Prints out the execution time for the policy iteration (difference between the end time and the start time)
print("Execution time of policy iteration (4 x 3) = ", end_time-start)

#Prints out the optimal policy list that is returned. 
optimal_policy = pi2
print("\nThis is the optimal policy determined by the policy iteration: \n", optimal_policy)

Computed utilities for this policy iteration:
[0.5094155954147008, 0.649586359613105, 0.7953622428927029, 1.0]
[0.3985112545104691, None, 0.4864404559151057, -1.0]
[0.2964665410943774, 0.2539605460927299, 0.34478839971672015, 0.12994247010553683]

This is the optimal policy in arrow/graph format: 
>   >      >   .
^   None   ^   .
^   >      ^   <
Execution time of policy iteration (4 x 3) =  0.0020008087158203125

This is the optimal policy determined by the policy iteration: 
 {(0, 1): (0, 1), (1, 2): (1, 0), (2, 1): (0, 1), (0, 0): (0, 1), (3, 1): None, (2, 0): (0, 1), (3, 0): (-1, 0), (0, 2): (1, 0), (2, 2): (1, 0), (1, 0): (1, 0), (3, 2): None}


In [5]:
#Pulls terminal states for the environment from the grid.
terminals = sequential_decision_environment.terminals

#TEST: print the list of terminal states in the environment.
#print("List of terminals: ", terminals, '\n')

#Initiates the starting state and the state variable
#THIS STARTING STATE CAN BE CHANGED THERE WILL BE AN ERROR THROWN IF YOU INPUT THE LOCATION OF THE OBSTACLE
starting_state = (0,0)
state = starting_state

#Prints the initial start point for the environment.
print("Start point in the environment: ", starting_state)

#Initiates the cumulative utility, move, and the end variable that will serve as the end of the while loop
move=0
utility_cumulative = 0
end = 0

#Discount value for reward which is the gamma value of the MDP (previously generated in original initiation code)
discount = sequential_decision_environment.gamma

#While loop that is used for the simulation of the navigation. 
while end != 1:
    
    #TEST prints out the current state before any manipulation.
    print("Current position: ", state)
    
    #TEST obtains the reward of the state from the environment and prints it.
    #reward = sequential_decision_environment.R(state)
    #print("Reward for this position = ", reward)
    
    #Calculates the utility for the current move given the move and the state reward. 
    utility_calc = (discount**move) * reward
    
    #Adds the current utility for the move to the cumulative utility for the trial and prints the 
    #current cumulative utility.
    utility_cumulative += utility_calc
    print("Utility for move =", utility_cumulative)
    
    #Changes to the next move
    move+=1
    
    #Checks if the current state is a terminal state. If so the navigation is terminated. If not
    #the navigation continues. 
    if state in terminals:
        end = 1
    else:
        #The possible new states moves for the current state are returned. 
        #This means that the states that can be moved to from the current state by only moving a single unit over
        #are captured in this variable.
        newstate = sequential_decision_environment.calculate_T(state, optimal_policy[state])
        
        #TEST: Prints the new state options availble for the current state to check that this is
        #being performed properly. 
        print("NEWSTATE OPTIONS: ", newstate)
        
        #Chooses a random direction from the choices 'ahead', 'turn_right', and 'turn_left' and prints the selected direction. 
        #These directions are with respect to the newstates possible. So if I am moving from (0,2) ahead would mean move to (1,2),
        #turn_right would mean to move to (0,1), and turn_left would mean to stay in the current state since a wall would be hit.
        direction_to_move =random.choices(['ahead', 'turn_right','turn_left'], k=1, weights = [0.8, 0.1, 0.1])
        print(direction_to_move)
        
        #Using the direction chosen randomly, the appropriate state coordinates are obtained from the new state options.
        #The 'ahead' direction would be represented in the first option ([0]), the 'turn_right' direction would be represented
        #in the second option [1], and the 'turn_left' direction would be represented in the third option [2]. 
        if direction_to_move == ['ahead']:
            newstate_coord = newstate[0][1]
        elif direction_to_move == ['turn_right']:
            newstate_coord = newstate[1][1]
        elif direction_to_move == ['turn_left']:
            newstate_coord = newstate[2][1]
            
        #The newstate coordinate is returned as well as the direction the move is occuring.
        print("NewState coord: ",direction_to_move, " , ", newstate_coord)
        
        #Disects the newstate coordinate into the x axis and y axis coor respectively.
        direction_coord_x = newstate_coord [0]
        direction_coord_y = newstate_coord [1]
        
        #Disects the current state coordinate into the x axis and y axis coor respectively.
        state_coord_x = state[0]
        state_coord_y = state[1]
       
        #Takes the difference between the x coordinates for the current state and the newstate as well as the y coord.
        #This is done to determine the amount and which axis the state needs to move. Without this, the amount of the move 
        #cannot be determined. DIRECTION VECTOR DETERMINATION
        new_direction_coord_1 = direction_coord_x - state_coord_x
        new_direction_coord_2 = direction_coord_y - state_coord_y
        
        #TEST: returns the direction vector
        direction_vector=[new_direction_coord_1,new_direction_coord_2]
        print("Direction vector = ", [new_direction_coord_1,new_direction_coord_2], "\n")
        
        #TEST: returns the current state before the change is made.
        print("State before change = ", state)
        
        #Performs the move from the current state to the newstate coordinate using the determined direction vector. 
        state = sequential_decision_environment.go(state, direction_vector)
        
        #TEST: returns the current state after the change is made.
        print("State after change = ", state, '\n') 
        
        #TEST: Troubleshooting using the optimal policy states to ensure that the code is working properly.
        #IF THIS IS USED IT WILL BE IDENTICAL TO THE POLICY UTILITIES SINCE IT IS ONLY USING THIS TO DO THE MOVES NO RANDOM ELEMENT
        #state = sequential_decision_environment.go(state, optimal_policy[state])

#The final cumulative utility for the trial is returned. 
print ("Final cumulative utility for trial = ", utility_cumulative)

Start point in the environment:  (0, 0)
Current position:  (0, 0)
Utility for move = 1.0
NEWSTATE OPTIONS:  [(0.8, (0, 1)), (0.1, (1, 0)), (0.1, (0, 0))]
['ahead']
NewState coord:  ['ahead']  ,  (0, 1)
Direction vector =  [0, 1] 

State before change =  (0, 0)
State after change =  (0, 1) 

Current position:  (0, 1)
Utility for move = 1.9
NEWSTATE OPTIONS:  [(0.8, (0, 2)), (0.1, (0, 1)), (0.1, (0, 1))]
['ahead']
NewState coord:  ['ahead']  ,  (0, 2)
Direction vector =  [0, 1] 

State before change =  (0, 1)
State after change =  (0, 2) 

Current position:  (0, 2)
Utility for move = 2.71
NEWSTATE OPTIONS:  [(0.8, (1, 2)), (0.1, (0, 1)), (0.1, (0, 2))]
['ahead']
NewState coord:  ['ahead']  ,  (1, 2)
Direction vector =  [1, 0] 

State before change =  (0, 2)
State after change =  (1, 2) 

Current position:  (1, 2)
Utility for move = 3.439
NEWSTATE OPTIONS:  [(0.8, (2, 2)), (0.1, (1, 2)), (0.1, (1, 2))]
['ahead']
NewState coord:  ['ahead']  ,  (2, 2)
Direction vector =  [1, 0] 

State befo

# BELOW ARE THE SAMPLE ENVIRONMENTS MADE FOR THE EXECUTION TIMING EXPERIMENTS.

There is a 6x5, 8x6, 12x9, and the original 4x3 environment here. Each of these are tested for execution time for the policy_iteration step only. 

In [6]:
#4x3
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, +1],
                                           [ -0.04, None, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(3, 2), (3, 1)])

start = time.time()

pi2 = policy_iteration(sequential_decision_environment)
end_time = time.time()

print("\nThis is the optimal policy in arrow/graph format: ")
print_table(sequential_decision_environment.to_arrows(pi2))

print("Execution time of policy iteration (4 x 3) = ", end_time-start)

Computed utilities for this policy iteration:
[0.5094155954147008, 0.649586359613105, 0.7953622428927029, 1.0]
[0.3985112545104691, None, 0.4864404559151057, -1.0]
[0.2964665410943774, 0.2539605460927299, 0.34478839971672015, 0.12994247010553683]

This is the optimal policy in arrow/graph format: 
>   >      >   .
^   None   ^   .
^   >      ^   <
Execution time of policy iteration (4 x 3) =  0.002000570297241211


In [7]:
#6x5
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, -0.04, -0.04, +1],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -1],
                                           [-0.04, None, -0.04, -0.04, -0.04, -0.04],
                                           [ -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(5, 4), (5, 3)])

start = time.time()

pi3 = policy_iteration(sequential_decision_environment)
end_time = time.time()

print("\nThis is the optimal policy in arrow/graph format: ")
print_table(sequential_decision_environment.to_arrows(pi3))

print("Execution time of policy iteration (6 x 5) = ", end_time-start)

Computed utilities for this policy iteration:
[0.2768939980227997, 0.37870189615609606, 0.49651011848269505, 0.6344353865966916, 0.7954508331453506, 1.0]
[0.21453525520398306, 0.30146044660563237, 0.3892303274403844, 0.49568446598152155, 0.4873362018029895, -1.0]
[0.1395919313986193, None, 0.30121131462684114, 0.37618289503720714, 0.3575673468232874, 0.14254245524207035]
[0.07960162679693036, 0.1325698864244548, 0.21339131323570956, 0.2731986325131458, 0.2571303321682535, 0.1677107281033714]
[0.026714310341025053, 0.07774279018381222, 0.13724256845704105, 0.18448857567541205, 0.17148587894237213, 0.10831186633825407]

This is the optimal policy in arrow/graph format: 
>   >      >   >   >   .
>   >      ^   ^   ^   .
^   None   ^   ^   ^   <
^   >      ^   ^   ^   <
^   >      ^   ^   ^   <
Execution time of policy iteration (6 x 5) =  0.006999492645263672


In [8]:
#8x6
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, +1],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -1],
                                           [-0.04, -0.04, -0.04, None, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(7, 5), (7, 4)])


start = time.time()

pi4 = policy_iteration(sequential_decision_environment)
end_time = time.time()

print("\nThis is the optimal policy in arrow/graph format: ")
print_table(sequential_decision_environment.to_arrows(pi4))

print("Execution time of policy iteration (8 x 6) = ", end_time-start)

Computed utilities for this policy iteration:
[0.11233370777620846, 0.18867518127787536, 0.27682995714021963, 0.37870189615609606, 0.49651011848269505, 0.6344353865966916, 0.7954508331453506, 1.0]
[0.07086159506977144, 0.13752050913231587, 0.2138877307245626, 0.30146044660563237, 0.3892303274403844, 0.49568446598152155, 0.4873362018029895, -1.0]
[0.0192977199405027, 0.0726730743958002, 0.1324612558431946, None, 0.30121129265109003, 0.37618267283794565, 0.35756489991835544, 0.14251548960875363]
[-0.027098355514138597, 0.016068201388577043, 0.06869377577637073, 0.1319503716033199, 0.2133348452319306, 0.2731909008760787, 0.2571026706876008, 0.1674306851948642]
[-0.06868207571791743, -0.03322081036805532, 0.0154607938531309, 0.0719305510225687, 0.13667137335317817, 0.18440594660174361, 0.1712008263242333, 0.10544853594448675]
[-0.10515392744899849, -0.06932199401875107, -0.027907141144347426, 0.018351430710920456, 0.06975840604848188, 0.10781542744679931, 0.09738765943534011, 0.04910751124

In [9]:
#12x9
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, +1],
                                           [-0.04, -0.04, -0.04, -0.40, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, None, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04],
                                           [-0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(11, 8), (11, 7)])


start = time.time()

pi5 = policy_iteration(sequential_decision_environment)
end_time = time.time()

print("\nThis is the optimal policy in arrow/graph format: ")
print_table(sequential_decision_environment.to_arrows(pi5))

print("Execution time of policy iteration (12 x 9) = ", end_time-start)

Computed utilities for this policy iteration:
[-0.1364786275489372, -0.0962772241071054, -0.04925625742954484, 0.008272862782585828, 0.11068052888687419, 0.1870926819592859, 0.2754339681291945, 0.3777316615380224, 0.4964146319286437, 0.6344260525335632, 0.795449993079669, 1.0]
[-0.1652883301368539, -0.1349752065337761, -0.11977394960370844, -0.3573519518488478, 0.06680611418188503, 0.1326875947770015, 0.20753460766878684, 0.2924141890108545, 0.3883395247877812, 0.4955968087575647, 0.48732770780554263, -1.0]
[-0.17714776979223334, -0.146363944267785, -0.11125468401511698, -0.06885457455819735, 0.014553314918058557, 0.07138150976821381, 0.13563704943715638, 0.20904312916922155, 0.2921953181340255, 0.3752997629065885, 0.3574720218786628, 0.1424343733008782]
[-0.18766422452265413, -0.15680892432067692, -0.12110816520968026, -0.07968095147559065, -0.031710248793611555, 0.019788340483678647, 0.07164313050140045, 0.13559449181240868, 0.20706174517159007, 0.27197346502194364, 0.256911985152853